# Загрузка данных

Может возникать ошибка с загрузкой бибилиотеки eli5 поэтому её функционал закомментирован.

In [ ]:
!pip install -U spacy
# !pip install eli5
!python3 -m spacy download en_core_web_sm

In [5]:
import re
import json
import spacy
import warnings
# import eli5
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from zipfile import ZipFile
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import classification_report

In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d uciml/sms-spam-collection-dataset

In [ ]:
ZIP_PATH = input("\nУкажите полный путь к .zip файлу:\n")

with ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall()

In [12]:
USERNAME = input("Укажите ваш ник в Kaggle:\n")
KEY = input("\nУкажите ваш токен в Kaggle:\n")

In [ ]:
api_token = {"username": USERNAME, "key": KEY}

with open('/root/.kaggle/kaggle.json', 'w+') as file:
    json.dump(api_token, file)

In [11]:
np.random.seed(42)
warnings.simplefilter("ignore", UserWarning)
pd.set_option('max_colwidth', 400)
plt.rcParams["figure.figsize"] = (12, 8)

# Набор данных
[
SMS Spam Collection Dataset](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset)

SMS Spam Collection - это множество СМС сообщений, которые были получены согласно исследованию "SMS Spam". Оно содержит одно множество из 5574 английских СМС сообщений, отмеченные как **ham** (не спам) или **spam** (спам).

In [15]:
FILE_PATH = input("\nУкажите полный путь к .csv файлу:\n")
data = pd.read_csv(FILE_PATH, encoding='iso-8859-1')[['v1', 'v2']].rename(columns={'v1': 'label', 'v2': 'text'})
data['label'] = data['label'].map({'ham': 0, 'spam': 1})

# Очистка

In [16]:
nlp = spacy.load("en_core_web_sm")
stopwords = nlp.Defaults.stop_words

data['cleaned_text'] = data['text'].apply(
    lambda x: ' '.join(
        token.lemma_.lower() for token in nlp(x) if
        not token.is_stop
        and not token.is_punct
        and not token.is_digit
        and token.is_ascii
        and not token.like_email
        and not token.like_num
        and not token.like_url
        and not token.is_space
    )
)

X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['label'], test_size=0.1, shuffle=True, random_state=42)

# TfidfTransformer vs  TfidfVectorizer vs CountVectorizer

In [ ]:
def custom_tokenize(text):
  text = re.sub(r'[^a-zA-Z ]', '', text)
  return text.split()

# CountVectorizer
custom_vectorizer = CountVectorizer(
    max_df=0.7,
    min_df=0.003,
    tokenizer=custom_tokenize,
)
pipe = Pipeline(
    steps=[
        ('counter', CountVectorizer()),
        ('clf', LogisticRegression())
    ]
).fit(X_train, y_train)
preds = pipe.predict(X_test)
print(f"CountVectorizer:\n{classification_report(y_test, preds)}\n")

# TfidfTransformer
pipe = Pipeline(
    steps=[
        ('counter', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', LogisticRegression())
    ]
).fit(X_train, y_train)
preds = pipe.predict(X_test)
print(f"TfidfTransformer:\n{classification_report(y_test, preds)}\n")

# TfidfVectorizer
pipe = Pipeline(
    steps=[
        ('tfidf', TfidfVectorizer()),
        ('clf', LogisticRegression())
    ]
).fit(X_train, y_train)
preds = pipe.predict(X_test)
print(f"TfidfVectorizer:\n{classification_report(y_test, preds)}")

Судя по результатам, CountVectorizer лучше подходит в качестве векторизации для данного датасета (проверено не только для логистической регрессии), т.к. имеет лучшие f1-меру, TP, TN, FP, FN, взвешенное среднее и макросреднее.

# Логистическая регрессия

In [18]:
def plot_grid_search(grid_search):
  '''
  Отрисовка графика подбора гиперпараметров
  '''
  results = pd.DataFrame(grid_search.cv_results_)
  results["params_str"] = results.params.apply(str)
  results.drop_duplicates(subset=("params_str", "iter"), inplace=True)
  mean_scores = results.pivot(
      index="iter",
      columns="params_str",
      values="mean_test_score"
  )
  ax = mean_scores.plot(legend=False, alpha=0.6)
  labels = [
      f"iter={i}\nn_samples={grid_search.n_resources_[i]}\nn_candidates={grid_search.n_candidates_[i]}"
      for i in range(grid_search.n_iterations_)
  ]
  ax.set_xticks(range(grid_search.n_iterations_))
  ax.set_xticklabels(labels, rotation=45, multialignment="left")
  ax.set_title("Scores of candidates over iterations")
  ax.set_ylabel("Mean test score", fontsize=15)
  ax.set_xlabel("Iterations", fontsize=15)
  plt.tight_layout()
  plt.grid()
  plt.show()

In [ ]:
pipe_log = Pipeline(
    steps=[
        ('counter', CountVectorizer()),
        ('clf', LogisticRegression())
    ]
)

parameter_grid = {
    "counter__max_df": [0.3, 0.4, 0.5, 0.6, 0.7],
    "counter__min_df": [0.001, 0.003, 0.005],
    "counter__ngram_range": ((1, 1), (1, 2)),
    "clf__solver": ["lbfgs", "liblinear"],
    "clf__C": np.linspace(0.1, 1, 10)
}

grid_search = HalvingGridSearchCV(
    pipe_log,
    param_grid=parameter_grid,
    n_jobs=-1,
    verbose=1,
    cv=2,
    scoring='accuracy',
    random_state=42,
).fit(X_train, y_train)
print(f"\nЛучшие параметры для логистической регрессии: {grid_search.best_params_}\n")

# Отрисовка графика
plot_grid_search(grid_search)

print(f"\nЛучший результат для логистической регрессии: {grid_search.best_score_}\n")

# eli5.show_weights(
#     estimator=grid_search.best_estimator_['clf'],
#     feature_names=list(grid_search.best_estimator_['counter'].get_feature_names_out()),
#     top=(20, 5)
# )

# Решающие деревья

In [ ]:
pipe_tree = Pipeline(
    steps=[
        ('counter', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', DecisionTreeClassifier())
    ]
)

parameter_grid_tree = {
    "counter__max_df": [0.3, 0.4, 0.5, 0.6, 0.7],
    "counter__min_df": [0.001, 0.003, 0.005],
    "counter__ngram_range": ((1, 1), (1, 2)),
    "tfidf__norm": ("l1", "l2"),
    "clf__criterion": ["gini", "entropy", "log_loss"],
    "clf__splitter": ["best", "random"],
    "clf__max_depth": range(1, 6),
    "clf__min_samples_split": [2, 3]
}

grid_search_tree = HalvingGridSearchCV(
    pipe_tree,
    param_grid=parameter_grid_tree,
    n_jobs=-1,
    verbose=1,
    cv=2,
    scoring='accuracy',
    random_state=42,
).fit(X_train, y_train)
print(f"\nЛучшие параметры для решаюших деревьев: {grid_search_tree.best_params_}\n")

# Отрисовка графика
plot_grid_search(grid_search_tree)

print(f"\nЛучший результат для решаюших деревьев: {grid_search_tree.best_score_}\n")

# Naive Bayes

In [ ]:
pipe_NB = Pipeline(
    steps=[
        ('counter', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', ComplementNB())
    ]
)

parameter_grid_NB = {
    "counter__max_df": [0.3, 0.4, 0.5, 0.6, 0.7],
    "counter__min_df": [0.001, 0.003, 0.005],
    "counter__ngram_range": ((1, 1), (1, 2)),
    "tfidf__norm": ("l1", "l2"),
    "clf__alpha": [0, True],
    "clf__norm": [True, False]
}

grid_search_NB = HalvingGridSearchCV(
    pipe_NB,
    param_grid=parameter_grid_NB,
    n_jobs=-1,
    verbose=1,
    cv=2,
    scoring='accuracy',
    random_state=42,
).fit(X_train, y_train)
print(f"\nЛучшие параметры для Наивного Байеса: {grid_search_NB.best_params_}\n")

# Отрисовка графика
plot_grid_search(grid_search_NB)

print(f"\nЛучший результат для Наивного Байеса: {grid_search_NB.best_score_}\n")

In [ ]:
# Сравнение на отложенной выборке
log_preds = grid_search.best_estimator_.predict(X_test)
print(f"Лучший результат для логистической регрессии:\n{classification_report(y_test, log_preds)}")
tree_preds = grid_search_tree.best_estimator_.predict(X_test)
print(f"\nЛучший результат для решаюших деревьев:\n{classification_report(y_test, tree_preds)}")
nb_preds = grid_search_NB.best_estimator_.predict(X_test)
print(f"\nЛучший результат для Наивного Байеса:\n{classification_report(y_test, nb_preds)}")

По всем метрикам классификатор на деревьях и Наивный Байес значительно уступают логистической регрессии.